In [1]:
from joblib import Parallel, delayed
import multiprocessing
  

In [2]:
# what are your inputs, and what operation do you want to 
# perform on each input. For example...
inputs = range(10) 
def processInput(i):
    return i * i
 
num_cores = multiprocessing.cpu_count()
     
results = Parallel(n_jobs=num_cores)(delayed(processInput)(i) for i in inputs)

print(results)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]


In [3]:
import numpy as np
d = {"a": 1, "b": 2, "c": 3, "d": 4}

def test_function(val):
    return [np.ones((2,2))*val, val]

num_cores = multiprocessing.cpu_count()
     
results = Parallel(n_jobs=num_cores)(delayed(test_function)(val) for val in list(d.values()))

print(results)
print(type(results))
print(len(results))
print(results[0])

[[array([[1., 1.],
       [1., 1.]]), 1], [array([[2., 2.],
       [2., 2.]]), 2], [array([[3., 3.],
       [3., 3.]]), 3], [array([[4., 4.],
       [4., 4.]]), 4]]
<class 'list'>
4
[array([[1., 1.],
       [1., 1.]]), 1]


# XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
# XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX

In [4]:
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.insert(1, "../python/functions")
sys.path.insert(2, "../python/architecture")

from data_prep_functions import mnist_prep
from model_functions import *
from plotting_functions import *
import no_gpu
import reproducible

X_train, X_val, y_train, y_val = mnist_prep()

# use samller dataset for increased speed
X_train_small = X_train[:1000, :]
X_val_small = X_val[:500, :]
y_train_small = y_train[:1000]
y_val_small = y_val[:500]

n_cols = X_train_small.shape[1]

# EnKF

X_train = X_train_small
X_test = X_val_small
y_train = y_train_small
y_test = y_val_small

batch_size = 50     # len(X_train)
epochs = 10
particles = 50
early_stopping = False
early_stopping_diff = 0.001
batch_normal = False # evtl. noch einbauen, obwohl im Paper nicht gemacht (aber Achtung mit den Dimensionen unten!!!)
shuffle = True
randomization = False

layers = 5
neurons = [128, 128, 64, 32, 10]
n_cols = X_train.shape[1]

delta = 0.005
h_0 = 2
epsilon = 0.5

n = len(X_train)
num_batches = int(np.ceil(n / batch_size))
batch_indices = np.cumsum([0] + list(np.ones(num_batches) * batch_size))
batch_indices[-1] = n

model_dict = {}
weights_dict = {}
y_pred_dict = {}
jacobian_dict = {}
weights_vector_dict = {}
train_acc_dict = {}
test_acc_dict = {}
iteration_dict = {}

# init_model already has weights and biases following the Glorot distribution
# it can already be used to predict and evaluate, but it is very bad (<10% accuracy)
# only used to determine shapes and shape_elements via its weights
init_model = nn_model_structure(layers = layers,
                                neurons = neurons,
                                n_cols = n_cols)
init_model = nn_model_compile(init_model,
                              optimizer = "sgd")
weights = init_model.get_weights()
# shape contains the shapes of the weight matrices and bias vectors as a list of arrays
shapes = [np.array(params.shape) for params in weights]
# shape_elements contains the indices of the weights as a vector and tells where to cut
shape_elements = np.cumsum([0] + [np.prod(shape) for shape in shapes])

for i in range(particles):
    # just an initial model with the correct structure regarding neurons, layers, activation functions, Glorot initialization
    model = nn_model_structure(layers = layers,
                               neurons = neurons,
                               n_cols = n_cols)
    model = nn_model_compile(model,
                             optimizer = "sgd")
    # for every particle write the model in a dictionary
    model_dict["model_{}".format(str(i+1))] = model
    
    # for every particles write the weights and biases in a dictionary
    weights_dict["model_{}".format(str(i+1))] = model_dict["model_{}".format(str(i+1))]\
                                                    .get_weights()
    
    train_acc_dict["model_{}".format(str(i+1))] = []
    test_acc_dict["model_{}".format(str(i+1))] = []
    iteration_dict["model_{}".format(str(i+1))] = []

In [5]:
def predict_function(model, batch):
    return model.predict(batch)

In [6]:
list(model_dict.values())[0].predict(X_test)

array([[0.10838139, 0.08183447, 0.10900812, ..., 0.04664213, 0.07889169,
        0.11503458],
       [0.03725751, 0.06413374, 0.2979478 , ..., 0.04504843, 0.02094523,
        0.2218028 ],
       [0.10595881, 0.04892114, 0.09535666, ..., 0.04889838, 0.05527588,
        0.13485011],
       ...,
       [0.07289157, 0.08365306, 0.11067791, ..., 0.05995197, 0.06411313,
        0.14524467],
       [0.13053434, 0.08308029, 0.13707955, ..., 0.06505534, 0.06666206,
        0.123865  ],
       [0.12124199, 0.06473883, 0.1367102 , ..., 0.06252323, 0.06677867,
        0.1086998 ]], dtype=float32)

In [8]:
num_cores = multiprocessing.cpu_count()
     
results = Parallel(n_jobs=num_cores, backend="threading")(delayed(predict_function)(model, X_test) for model in list(model_dict.values()))

print(results)

[array([[0.10838139, 0.08183447, 0.10900812, ..., 0.04664213, 0.07889169,
        0.11503458],
       [0.03725751, 0.06413374, 0.2979478 , ..., 0.04504843, 0.02094523,
        0.2218028 ],
       [0.10595881, 0.04892114, 0.09535666, ..., 0.04889838, 0.05527588,
        0.13485011],
       ...,
       [0.07289157, 0.08365306, 0.11067791, ..., 0.05995197, 0.06411313,
        0.14524467],
       [0.13053434, 0.08308029, 0.13707955, ..., 0.06505534, 0.06666206,
        0.123865  ],
       [0.12124199, 0.06473883, 0.1367102 , ..., 0.06252323, 0.06677867,
        0.1086998 ]], dtype=float32), array([[0.05636015, 0.09261063, 0.12620185, ..., 0.13124913, 0.06503075,
        0.07928127],
       [0.07231994, 0.05591666, 0.06145753, ..., 0.10196716, 0.05173903,
        0.03629709],
       [0.07982113, 0.08016572, 0.11946279, ..., 0.15317361, 0.05661031,
        0.0776928 ],
       ...,
       [0.08179331, 0.07704432, 0.09883907, ..., 0.16131754, 0.05463371,
        0.10048045],
       [0.05060117

In [9]:
mean_weights = list(np.mean(list(weights_dict.values()), axis = 0))
mean_model = init_model
mean_model.set_weights(mean_weights)

mean_model_train_acc = np.array(mean_model.evaluate(X_train, y_train)[1])
mean_model_test_acc = np.array(mean_model.evaluate(X_test, y_test)[1])

16/16 [==============================] - 0s 1ms/step - loss: 2.3061 - accuracy: 0.0860


In [10]:
import time
start_time = time.time()

In [11]:
# loop over all epochs
for epoch in range(epochs):
    # early stopping
    if early_stopping:
        if epoch == 0:
            test_acc_old = 0
        else:
            test_acc_new = mean_model_test_acc[epoch]
            if np.absolute(test_acc_new - test_acc_old) <= early_stopping_diff:
                print("STOP: Early Stopping after epoch {} because improvement in test accuracy is only {}."\
                                                                     .format(epoch+1, test_acc_new - test_acc_old))
                break
            test_acc_old = test_acc_new
    # shuffle the data
    if shuffle:
        indices = y_train.sample(frac=1).index
        X_batches = [X_train[indices][int(batch_indices[i]):int(batch_indices[i+1])] for i in range(len(batch_indices)-1)]
        y_batches = [y_train.iloc[indices].reset_index(drop = True)[int(batch_indices[i]):int(batch_indices[i+1])] for i in range(len(batch_indices)-1)]   
    # loop over all batches
    for b in range(num_batches):    
        for i in range(particles):
            # set new weights for model
            model_dict["model_{}".format(str(i+1))].set_weights(weights_dict["model_{}".format(str(i+1))])
        
        y_preds = Parallel(n_jobs=num_cores, backend="threading")(delayed(predict_function)(model, X_batches[b]) for model in list(model_dict.values()))
        for i in range(particles):
            # for every particle write the predictions on the training batches in a dictionary
            y_pred_dict["model_{}".format(str(i+1))] = y_preds[i]
        
        for i in range(particles):

            # for every particle write the Jacobian in a dictionary
            jacobian_dict["model_{}".format(str(i+1))] = (-1) * np.multiply(np.array(y_batches[b]), 
                                                                            np.array(1 / (y_pred_dict["model_{}".format(str(i+1))] + delta)))
            
        # compute the mean of the predictions
        y_pred_mean = np.mean(list(y_pred_dict.values()), axis = 0)
        
        # compute the matrix D elementwise
        d = np.zeros(shape = (particles, particles))
        for k in range(particles):
            y_pred_centered = y_pred_dict["model_{}".format(str(k+1))] - y_pred_mean
            for j in range(particles):
                d[k][j] = np.sum(np.multiply(y_pred_centered, jacobian_dict["model_{}".format(str(j+1))]))
        d = np.transpose(d)  
        
        # compute the scalar h_t
        h_t = h_0 / (np.sqrt(np.sum(d**2)) + epsilon)
        
        # Reshape the weights and biases so that they are no longer matrices and vectores, but now one single vector
        for i in range(particles):
            weights_array = np.array([])
            for j in range(len(weights_dict["model_{}".format(str(i+1))])):
                weights_array = np.append(weights_array, np.reshape(weights_dict["model_{}".format(str(i+1))][j], (1, -1)).ravel())
            weights_vector_dict["model_{}".format(str(i+1))] = weights_array
          
        # matrix with particle parameters as row vectors
        weights_all_ptcls = np.array(list(weights_vector_dict.values()))

        # compute the matrix with the updates for each particle
        weights_all_ptcls = weights_all_ptcls - h_t * np.matmul(d, weights_all_ptcls)

        for i in range(particles):
            # write the updates back into the dictionary
            weights_vector_dict["model_{}".format(str(i+1))] = weights_all_ptcls[i]
            # reshape the updates, so that they are of the original matrx and vector shape
            for l in range(len(shape_elements)-1):
                start = shape_elements[l]
                end = shape_elements[l+1]
                weights_dict["model_{}".format(str(i+1))][l] = np.reshape(weights_vector_dict["model_{}".format(str(i+1))][start:end], tuple(shapes[l]))
            if randomization:
                # add randomization/ noise to each particle
                new_weights = []
                # standard deviation for scaled Glorot distribution
                for s in range(len(shapes)):
                    if shapes[s].shape[0] == 2:
                        fan_in = shapes[s][0]
                        fan_out = shapes[s][1]
                    if shapes[s].shape[0] == 1:
                        fan_in = shapes[s-1][0]
                        fan_out = shapes[s][0]
                    stddev = np.sqrt(np.sqrt(h_t)) * np.sqrt(2 / (fan_in + fan_out))
                    noise = np.random.normal(loc = 0.0,
                                             scale = stddev,
                                             size = tuple(shapes[s]))
                    new_weights.append(weights_dict["model_{}".format(str(i+1))][s] + noise)
                weights_dict["model_{}".format(str(i+1))] = new_weights
                
    if randomization:
        # randomize particles around their mean
        weights_mean = list(np.mean(list(weights_dict.values()), axis = 0))
        for i in range(particles):
            new_weights = []
            # standard deviation for Glorot distribution
            for s in range(len(shapes)):
                if shapes[s].shape[0] == 2:
                    fan_in = shapes[s][0]
                    fan_out = shapes[s][1]
                if shapes[s].shape[0] == 1:
                    fan_in = shapes[s-1][0]
                    fan_out = shapes[s][0]
                stddev = np.sqrt(2 / (fan_in + fan_out))
                noise = np.random.normal(loc = 0.0,
                                         scale = stddev,
                                         size = tuple(shapes[s]))
                new_weights.append(weights_mean[s] + noise)
            weights_dict["model_{}".format(str(i+1))] = new_weights
            
    for i in range(particles):
        # for every particle write the training accuracy of the current iteration in a dictionary
        train_acc_dict["model_{}".format(str(i+1))].append(model_dict["model_{}".format(str(i+1))]\
                                                                  .evaluate(X_train, y_train, verbose = 0)[1])

        # for every particle write the test accuracy of the current iteration in a dictionary
        test_acc_dict["model_{}".format(str(i+1))].append(model_dict["model_{}".format(str(i+1))]\
                                                                  .evaluate(X_test, y_test, verbose = 0)[1])

        # for every particle write the current iteration in a dictionary
        iteration_dict["model_{}".format(str(i+1))].append("Epoch: {}, Batch: {}.".format(epoch+1, b+1))
            
    # update the mean_model
    mean_weights = list(np.mean(list(weights_dict.values()), axis = 0))
    mean_model.set_weights(mean_weights)
    
    mean_model_train_acc = np.append(mean_model_train_acc, np.array(mean_model.evaluate(X_train, y_train, verbose = 0)[1]))
    mean_model_test_acc = np.append(mean_model_test_acc, np.array(mean_model.evaluate(X_test, y_test, verbose = 0)[1]))
    
    print("Training Accuracy after Epoch {}: {}".format(str(epoch+1), str(np.round(mean_model_train_acc[-1], 3))))

Training Accuracy after Epoch 1: 0.111
Training Accuracy after Epoch 2: 0.204
Training Accuracy after Epoch 3: 0.209
Training Accuracy after Epoch 4: 0.226
Training Accuracy after Epoch 5: 0.229
Training Accuracy after Epoch 6: 0.246
Training Accuracy after Epoch 7: 0.256
Training Accuracy after Epoch 8: 0.26
Training Accuracy after Epoch 9: 0.269
Training Accuracy after Epoch 10: 0.27


In [12]:
end_time = time.time()
print("Calculation time: {} minutes.".format((end_time - start_time) / 60))

Calculation time: 5.246686553955078 minutes.


In [13]:
print(np.round(mean_model_train_acc, 3))
print(np.round(mean_model_test_acc, 3))

[0.094 0.111 0.204 0.209 0.226 0.229 0.246 0.256 0.26  0.269 0.27 ]
[0.086 0.112 0.16  0.162 0.182 0.198 0.2   0.212 0.21  0.206 0.21 ]


In [15]:
def predict_function(model, batch):
    return model.predict(batch)

In [16]:
# Parallelizing using Pool.apply()

import multiprocessing as mp

# Step 1: Init multiprocessing.Pool()
pool = mp.Pool(mp.cpu_count())

# Step 2: `pool.apply` the `howmany_within_range()`
results = [pool.apply(predict_function, args=(model, X_test)) for model in list(model_dict.values())]

# Step 3: Don't forget to close
pool.close()    

print(results[:10])

TypeError: can't pickle _thread.RLock objects